In [19]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import mode
from collections import defaultdict

In [13]:
train_fname = 'data/train.csv'
test_fname = 'data/test.csv'
df = pd.read_csv(train_fname, sep=';')
df_test = pd.read_csv(test_fname, sep=';')
y = df['VARIABLE_CIBLE'].values == 'GRANTED'
df = df.drop('VARIABLE_CIBLE', axis=1)
df = pd.concat((df, df_test)).reset_index()
df = df.drop('index', axis=1)

In [14]:
y

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)

##### Remarque : Beaucoup de features à priori continues mais à valeurs discrêtes sont mises dans les catégories car elles ne prennent pas beaucoup de valeurs différentes.

In [3]:
f_con = ['cited_age_min','cited_age_median','cited_age_max','cited_age_mean',
        'pct_NB_IPC','pct_NB_IPC_LY','oecd_NB_BACKWARD_PL','oecd_NB_BACKWARD_NPL',
        'IDX_ORIGIN','IDX_RADIC']
f_cat = ['APP_NB','APP_NB_PAYS','APP_NB_TYPE','VOIE_DEPOT','COUNTRY','SOURCE_BEGIN_MONTH','FISRT_APP_COUNTRY',
         'FISRT_APP_TYPE','LANGUAGE_OF_FILLING','FIRST_CLASSE','TECHNOLOGIE_SECTOR','TECHNOLOGIE_FIELD','MAIN_IPC',
         'FISRT_INV_COUNTRY','FISRT_INV_TYPE','SOURCE_CITED_AGE','SOURCE_IDX_ORI','SOURCE_IDX_RAD',
         'NB_CLASSES','NB_ROOT_CLASSES','NB_SECTORS','NB_FIELDS','INV_NB','INV_NB_PAYS','INV_NB_TYPE',
         'NB_BACKWARD_NPL','NB_BACKWARD_XY','NB_BACKWARD_I','NB_BACKWARD_AUTRE','NB_BACKWARD_PL',
         'NB_BACKWARD','oecd_NB_ROOT_CLASSES','PRIORITY_MONTH','FILING_MONTH','PUBLICATION_MONTH','BEGIN_MONTH','cited_n']

##### LabelEncoder permet de n'encoder qu'une colonne à la fois. Pour éviter la boucle for, on a trouvé cette classe sur StackOverflow :

In [4]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [5]:
df_cat = MultiColumnLabelEncoder().fit_transform(df[f_cat].astype(str))

##### FIRST_CLASS possède plus de 40000 valeurs possibles, ce qui pose un problème de taille pour le training. On choisit donc de ne garder que les 6000 valeurs plus fréquentes (valeur trouvée empiriquement). Le training prend quand même du temps mais comme on utilise plusieurs machines, on s'en sort.

In [6]:
S = set(df_cat['FIRST_CLASSE'].value_counts()[0:6000].reset_index()[['index']].values[:,0])
Y = df_cat.values
for i in range(df_cat.shape[0]):
    if(not Y[i][9] in S):
        Y[i][9] = 41178
df_cat = pd.DataFrame(Y, columns=f_cat)

##### On utilise Imputer pour combler les trous des features continues. Il y aura probablement mieux à faire en inférent les valeurs 

In [7]:
X = pd.DataFrame(Imputer().fit_transform(df[f_con].values), columns=f_con)
df_2 = pd.concat((X, df_cat), axis=1)
encoder = OneHotEncoder()
n_samples = 259431
df1 = df_cat[0:n_samples]
df2 = df_cat[n_samples:]

In [12]:
df1 = encoder.fit_transform(df1)
df2 = encoder.transform(df2)

ValueError: unknown categorical feature present [  9 120] during transform.

In [28]:
X_train = df1.values
X_test = df2.values

##### On a mis peu d'arbres pour le code s'éxécute en un temps raisonnable. En pratique on l'avait fait avec 8000 arbres.

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=36000, oob_score=False, min_samples_leaf=1,
                             min_samples_split=1, n_jobs=n_thread)
clf.fit(X_train,y)

In [ ]:
np.savetxt('subs/my_sub.txt', clf.predict_proba(X_test)[:,1])